### Author : Antonio Sirica - Course : Computational Intelligence

In [1]:

from random import random, seed
from itertools import product
import numpy as np
import copy
from tqdm.auto import tqdm
%pip install icecream
from icecream import ic
from matplotlib import pyplot as plt
from itertools import accumulate


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# Instance 1

In [2]:
UNIVERSE_SIZE = 100
NUM_SETS = 10
DENSITY = 0.2

rng = np.random.Generator(np.random.PCG64([UNIVERSE_SIZE, NUM_SETS, int(10_000 * DENSITY)])) #seed 
ic(rng)

ic| rng: Generator(PCG64) at 0x241FF570820


Generator(PCG64) at 0x241FF570820

# Instance 2

In [19]:


UNIVERSE_SIZE = 1_000
NUM_SETS = 100
DENSITY = 0.2

rng = np.random.Generator(np.random.PCG64([UNIVERSE_SIZE, NUM_SETS, int(10_000 * DENSITY)])) #seed 
ic(rng)

ic| rng: Generator(PCG64) at 0x241880475A0


Generator(PCG64) at 0x241880475A0

# Instance 3

In [4]:


UNIVERSE_SIZE = 10_000
NUM_SETS = 1_000
DENSITY = 0.2

rng = np.random.Generator(np.random.PCG64([UNIVERSE_SIZE, NUM_SETS, int(10_000 * DENSITY)])) #seed 
ic(rng)

ic| rng: Generator(PCG64) at 0x241FF5F2F80


Generator(PCG64) at 0x241FF5F2F80

# Instance 4

In [40]:


UNIVERSE_SIZE = 100_000
NUM_SETS = 10_000
DENSITY = 0.1

rng = np.random.Generator(np.random.PCG64([UNIVERSE_SIZE, NUM_SETS, int(10_000 * DENSITY)])) #seed 
ic(rng)

ic| rng: Generator(PCG64) at 0x24189537D80


Generator(PCG64) at 0x24189537D80

# Instance 5

In [6]:


UNIVERSE_SIZE = 100_000
NUM_SETS = 10_000
DENSITY = 0.2

rng = np.random.Generator(np.random.PCG64([UNIVERSE_SIZE, NUM_SETS, int(10_000 * DENSITY)])) #seed 
ic(rng)

ic| rng: Generator(PCG64) at 0x241FF5F2340


Generator(PCG64) at 0x241FF5F2340

# Instance 6

In [37]:
UNIVERSE_SIZE = 100_000
NUM_SETS = 10_000
DENSITY = 0.3

rng = np.random.Generator(np.random.PCG64([UNIVERSE_SIZE, NUM_SETS, int(10_000 * DENSITY)])) #seed 
ic(rng)

ic| rng: Generator(PCG64) at 0x24189536960


Generator(PCG64) at 0x24189536960

# Execute from here with the chosen instance

In [41]:
# DON'T EDIT THESE LINES!

SETS = np.random.random((NUM_SETS, UNIVERSE_SIZE)) < DENSITY # Randomly generate sets an array of size NUM_SETS x UNIVERSE_SIZE with values True or False depending on if < DENSITY
ic(SETS)
for s in range(UNIVERSE_SIZE):
    if not np.any(SETS[:, s]): # if no subset covers the element s
        SETS[np.random.randint(NUM_SETS), s] = True # a random subset now covers that element
# Define the cost for each set based on the sum of elements in the set, raised to the power of 1.1
COSTS = np.power(SETS.sum(axis=1), 1.1) # element 0 cost of subset/row 0 ...




ic| SETS: array([[False,  True, False, ..., False, False, False],
                 [False, False, False, ..., False, False, False],
                 [False,  True, False, ..., False, False, False],
                 ...,
                 [False, False, False, ..., False, False, False],
                 [False, False, False, ..., False, False, False],
                 [False, False, False, ..., False, False, False]])


In [9]:
#solution is a vector of the subsets taken (where true) 


def valid(solution):
    """Checks wether solution is valid (ie. covers all universe)"""
    return np.all(np.logical_or.reduce(SETS[solution])) 


def cost(solution):
    """Returns the cost of a solution (to be minimized)"""
    return COSTS[solution].sum()



In [21]:
def coverage(solution):
    """Restituisce il numero di elementi nell'universo che sono coperti dai set selezionati."""
    covered_elements = np.sum(SETS[solution], axis=0) > 0
    return np.sum(covered_elements)


def fitness(solution):
    return (coverage(solution), -cost(solution), valid(solution))

def single_mutation (solution):
    i = rng.integers(0, NUM_SETS)
    new_solution = solution.copy()
    new_solution[i]=not new_solution[i]
    return new_solution

def multiple_mutation_strength(solution, strength:float = 0.1): # for larger steps
    mask = rng.random(NUM_SETS) < strength # Each element has a strength probability of being True and a remaining probability of being False.
    if not np.any(mask):
        mask[np.random.randint(NUM_SETS)] = True #if none is True put a random one to true

    new_solution = np.logical_xor(solution, mask)
    return new_solution


In [42]:

ITERATIONS = 1000
BUFFER_SIZE = 5 # 1 out of 5


solution = rng.random(NUM_SETS) < 1

solution_fitness = fitness(solution)
history = [float(solution_fitness[0])]
ic(fitness(solution))

strength = 0.5

buffer = list()

tweak = multiple_mutation_strength

bufferValidSolutions = list()


for steps in range(ITERATIONS):

    new_solution = tweak(solution, strength)

    if (valid(new_solution)):
            bufferValidSolutions.append((new_solution, cost(new_solution)))

    f = fitness(new_solution)
    history.append(float(f[0]))

    buffer.append(f > solution_fitness) #we insert a true if there was an improvement in fitness

    buffer = buffer[-BUFFER_SIZE:] #keep only the last BUFFER_SIZE elements of the buffer

    # SELF-ADAPTIVE
    
    if sum(buffer) > 1 :
        strength *= 1.1 # magical numbers (important is to understand correlations)
    elif sum(buffer) == 0 :
        strength /= 1.1

    if f > solution_fitness: #here twe tuple rapresentation makes implicitly the valid check
        solution = new_solution
        solution_fitness = fitness(new_solution)


ic(solution)
ic(fitness(solution))

best_solution = min(bufferValidSolutions, key=lambda x:x[1])

solution_with_min_cost, min_cost = best_solution


ic(valid(solution_with_min_cost))
ic("Number of set taken",np.sum(solution))
ic(min_cost)

#plt.figure(figsize=(14,8))

#plt.plot(
#    range(len(history)),
#    list(accumulate(history, max)),
#    color="red",
#)

#_ = plt.scatter(range(len(history)), history, marker=".")



ic| fitness(solution): (100000, -251188517.84379008, True)
ic| solution: array([False, False,  True, ...,  True, False, False])
ic| fitness(solution): (100000, -103421506.26222196, True)
ic| valid(solution_with_min_cost): True
ic| 'Number of set taken', np.sum(solution): 4117
ic| min_cost: 103421506.26222196


103421506.26222196